# Épica 2: Extracción - Testing y Validación

Este notebook valida la implementación de la Épica 2 usando PDFs existentes del proyecto.
**No requiere reportlab** - usa boletines reales.

**Tareas implementadas:**
- ✅ **2.3**: Modelo ExtractedContent unificado
- ✅ **2.1**: Extractores PyPDF2 + pdfplumber
- ✅ **2.2**: ExtractorRegistry con Strategy Pattern

**Fecha**: 2026-02-10

In [1]:
# Setup - Agregar path del backend
import sys
from pathlib import Path

backend_path = Path().absolute().parent / 'watcher-monolith' / 'backend'
if str(backend_path) not in sys.path:
    sys.path.insert(0, str(backend_path))

print(f"✅ Backend path: {backend_path}")
print(f"✅ Path agregado al sys.path")

✅ Backend path: /Users/germanevangelisti/watcher-agent/watcher-monolith/backend
✅ Path agregado al sys.path


In [2]:
# Imports
import asyncio
from datetime import datetime
import json

print("Importando módulos de la Épica 2...\n")

# Schemas de extracción
from app.schemas.extraction import (
    ExtractedContent,
    PageContent,
    ContentSection,
    ExtractionStats,
    ExtractionMethod,
    SectionType
)
print("✅ Schemas importados (extraction.py)")

# Extractores
from app.services.extractors import (
    PDFExtractor,
    PyPDF2Extractor,
    PdfPlumberExtractor,
    ExtractorRegistry,
    extract_pdf
)
print("✅ Extractores importados (extractors/)")

print("\n" + "="*70)
print("✅ TODOS LOS MÓDULOS IMPORTADOS CORRECTAMENTE")
print("="*70)

Importando módulos de la Épica 2...

✅ Schemas importados (extraction.py)
✅ Extractores importados (extractors/)

✅ TODOS LOS MÓDULOS IMPORTADOS CORRECTAMENTE


In [3]:
# Buscar PDFs existentes del proyecto
def find_test_pdfs(max_pdfs=5):
    """Busca PDFs en el directorio de boletines."""
    base_dir = Path().absolute().parent
    boletines_dir = base_dir / "boletines"
    
    pdfs = []
    
    if boletines_dir.exists():
        # Buscar PDFs recursivamente
        pdfs = list(boletines_dir.glob("**/*.pdf"))[:max_pdfs]
    
    return pdfs

# Buscar PDFs
test_pdfs = find_test_pdfs(max_pdfs=5)

print(f"🔍 PDFs encontrados: {len(test_pdfs)}\n")
if test_pdfs:
    for i, pdf in enumerate(test_pdfs, 1):
        size_kb = pdf.stat().st_size // 1024
        print(f"   {i}. {pdf.name:<35} ({size_kb:>6} KB)")
else:
    print("⚠️  No se encontraron PDFs en /boletines/")
    print("   El testing se limitará a validación de schemas")

🔍 PDFs encontrados: 5

   1. 20250327_2_Secc.pdf                 (  1228 KB)
   2. 20250306_5_Secc.pdf                 (   661 KB)
   3. 20250306_4_Secc.pdf                 (  2641 KB)
   4. 20250327_3_Secc.pdf                 (   719 KB)
   5. 20250331_4_Secc.pdf                 (  3617 KB)


---
## 📋 Tarea 2.3: Modelo ExtractedContent Unificado

Validación de schemas Pydantic que unifican la salida de extractores.

In [4]:
# Test 1: Crear ExtractedContent de ejemplo
print("🧪 Test 1: Creación de ExtractedContent\n")

test_content = ExtractedContent(
    success=True,
    source_path="/test/documento.pdf",
    full_text="Texto de prueba del documento.",
    pages=[
        PageContent(
            page_number=1,
            text="Texto de prueba.",
            char_count=18
        )
    ],
    sections=[
        ContentSection(
            section_type=SectionType.GENERAL,
            content="Texto de prueba.",
            start_page=1,
            end_page=1
        )
    ],
    stats=ExtractionStats(
        total_chars=18,
        total_pages=1,
        extraction_method=ExtractionMethod.PYPDF2
    ),
    extracted_at=datetime.utcnow()
)

print(f"✅ ExtractedContent creado exitosamente")
print(f"   - Type: {type(test_content).__name__}")
print(f"   - Success: {test_content.success}")
print(f"   - Páginas: {test_content.stats.total_pages}")
print(f"   - Método: {test_content.stats.extraction_method.value}")
print(f"   - Secciones: {len(test_content.sections)}")

🧪 Test 1: Creación de ExtractedContent

✅ ExtractedContent creado exitosamente
   - Type: ExtractedContent
   - Success: True
   - Páginas: 1
   - Método: pypdf2
   - Secciones: 1


In [5]:
# Test 2: Serialización a JSON
print("\n🧪 Test 2: Serialización JSON\n")

json_str = test_content.model_dump_json(indent=2)
json_dict = test_content.model_dump()

print(f"✅ Serializado a JSON string: {len(json_str)} caracteres")
print(f"✅ Serializado a dict: {len(json_dict)} campos")
print(f"\n📝 Preview del JSON:")
print("-" * 70)
print(json_str[:400] + "...")
print("-" * 70)


🧪 Test 2: Serialización JSON

✅ Serializado a JSON string: 631 caracteres
✅ Serializado a dict: 9 campos

📝 Preview del JSON:
----------------------------------------------------------------------
{
  "success": true,
  "source_path": "/test/documento.pdf",
  "full_text": "Texto de prueba del documento.",
  "pages": [
    {
      "page_number": 1,
      "text": "Texto de prueba.",
      "char_count": 18
    }
  ],
  "sections": [
    {
      "section_type": "general",
      "content": "Texto de prueba.",
      "start_page": 1,
      "end_page": 1,
      "metadata": {}
    }
  ],
  "stats": ...
----------------------------------------------------------------------


In [6]:
# Test 3: Tipos de sección
print("\n🧪 Test 3: Enums de tipos de sección\n")

section_types = list(SectionType)
extraction_methods = list(ExtractionMethod)

print(f"✅ SectionType - {len(section_types)} tipos:")
for st in section_types:
    print(f"   • {st.value}")

print(f"\n✅ ExtractionMethod - {len(extraction_methods)} métodos:")
for em in extraction_methods:
    print(f"   • {em.value}")


🧪 Test 3: Enums de tipos de sección

✅ SectionType - 7 tipos:
   • licitacion
   • nombramiento
   • resolucion
   • subsidio
   • presupuesto
   • general
   • unknown

✅ ExtractionMethod - 2 métodos:
   • pypdf2
   • pdfplumber


---
## 🎯 Tarea 2.2: ExtractorRegistry

Testing del registry antes de usar extractores individuales.

In [7]:
# Test 4: ExtractorRegistry - Inicialización
print("🧪 Test 4: ExtractorRegistry - Configuración\n")

extractors = ExtractorRegistry.list_extractors()
default = ExtractorRegistry.get_default()

print(f"✅ Extractores registrados: {extractors}")
print(f"✅ Extractor por defecto: {default}")

print(f"\n📋 Detalles de cada extractor:\n")
for name in extractors:
    extractor = ExtractorRegistry.get(name)
    print(f"   • {name}:")
    print(f"     - Clase: {type(extractor).__name__}")
    print(f"     - Método: {extractor.method.value}")

🧪 Test 4: ExtractorRegistry - Configuración

✅ Extractores registrados: ['pypdf2', 'pdfplumber']
✅ Extractor por defecto: pdfplumber

📋 Detalles de cada extractor:

   • pypdf2:
     - Clase: PyPDF2Extractor
     - Método: pypdf2
   • pdfplumber:
     - Clase: PdfPlumberExtractor
     - Método: pdfplumber


---
## 🔧 Tarea 2.1: Extractores Unificados

Testing con PDFs reales del proyecto.

In [8]:
# Test 5: PyPDF2Extractor con PDF real
print("🧪 Test 5: PyPDF2Extractor - Extracción de boletín real\n")

if test_pdfs:
    pdf_path = test_pdfs[0]
    print(f"📄 PDF: {pdf_path.name}")
    print(f"📂 Ubicación: {pdf_path.parent}")
    print(f"📊 Tamaño: {pdf_path.stat().st_size // 1024} KB\n")
    
    extractor = PyPDF2Extractor(calculate_tokens=False)
    result = await extractor.extract(pdf_path)
    
    print(f"✅ Extracción exitosa: {result.success}")
    if result.success:
        print(f"   - Método: {result.stats.extraction_method.value}")
        print(f"   - Páginas: {result.stats.total_pages}")
        print(f"   - Caracteres: {result.stats.total_chars:,}")
        print(f"   - Duración: {result.stats.extraction_duration_ms:.2f} ms")
        print(f"   - Velocidad: {result.stats.total_chars / result.stats.extraction_duration_ms * 1000:.0f} chars/seg")
        print(f"\n📝 Texto extraído (primeras 300 caracteres):")
        print("-" * 70)
        print(result.full_text[:300].strip())
        print("-" * 70)
    else:
        print(f"❌ Error: {result.error}")
else:
    print("⚠️  No hay PDFs disponibles. Coloca boletines en /boletines/")

🧪 Test 5: PyPDF2Extractor - Extracción de boletín real

📄 PDF: 20250327_2_Secc.pdf
📂 Ubicación: /Users/germanevangelisti/watcher-agent/boletines/2025/03
📊 Tamaño: 1228 KB

✅ Extracción exitosa: True
   - Método: pypdf2
   - Páginas: 79
   - Caracteres: 630,580
   - Duración: 2438.08 ms
   - Velocidad: 258638 chars/seg

📝 Texto extraído (primeras 300 caracteres):
----------------------------------------------------------------------
SUMARIOSECCION1
BOLETIN OFICIAL DE LA PROVINCIA DE CORDOBAJUEVES 27 DE MARZO DE 2025
AÑO CXII - TOMO DCCXXIII - Nº 61
CÓRDOBA, (R.A.) 
http://boletinoficial.cba.gov.ar
Email: boe@cba.gov.arJUDICIALES2aRemates  ....................................................  Pág. 1
Concursos y Quiebras  .......
----------------------------------------------------------------------


In [9]:
# Test 6: PdfPlumberExtractor con PDF real
print("\n🧪 Test 6: PdfPlumberExtractor - Extracción de boletín real\n")

if test_pdfs:
    pdf_path = test_pdfs[0]
    print(f"📄 PDF: {pdf_path.name}\n")
    
    try:
        extractor = PdfPlumberExtractor(calculate_tokens=False)
        result = await extractor.extract(pdf_path)
        
        print(f"✅ Extracción exitosa: {result.success}")
        if result.success:
            print(f"   - Método: {result.stats.extraction_method.value}")
            print(f"   - Páginas: {result.stats.total_pages}")
            print(f"   - Caracteres: {result.stats.total_chars:,}")
            print(f"   - Duración: {result.stats.extraction_duration_ms:.2f} ms")
            print(f"   - Velocidad: {result.stats.total_chars / result.stats.extraction_duration_ms * 1000:.0f} chars/seg")
            print(f"\n📝 Texto extraído (primeras 300 caracteres):")
            print("-" * 70)
            print(result.full_text[:300].strip())
            print("-" * 70)
        else:
            print(f"❌ Error: {result.error}")
    except ImportError:
        print("⚠️  pdfplumber no está instalado")
        print("   Para instalar: pip install pdfplumber")
else:
    print("⚠️  No hay PDFs disponibles")


🧪 Test 6: PdfPlumberExtractor - Extracción de boletín real

📄 PDF: 20250327_2_Secc.pdf

✅ Extracción exitosa: True
   - Método: pdfplumber
   - Páginas: 79
   - Caracteres: 616,325
   - Duración: 19142.90 ms
   - Velocidad: 32196 chars/seg

📝 Texto extraído (primeras 300 caracteres):
----------------------------------------------------------------------
2
a
JUDICIALES
JUEVES 27 DE MARZO DE 2025
AÑO CXII - TOMO DCCXXIII - Nº 61 SECCION
CÓRDOBA, (R.A.)
http://boletinoficial.cba.gov.ar
Email: boe@cba.gov.ar SUMARIO
REMATES Remates .................................................... Pág. 1
las sumas entregadas a favor de la vendedora.
Concursos y Quie
----------------------------------------------------------------------


In [10]:
# Test 7: Comparación de velocidad y calidad
print("\n🧪 Test 7: Comparación PyPDF2 vs pdfplumber\n")

if test_pdfs:
    pdf_path = test_pdfs[0]
    print(f"📄 Comparando con: {pdf_path.name}\n")
    
    # PyPDF2
    extractor1 = PyPDF2Extractor(calculate_tokens=False)
    result1 = await extractor1.extract(pdf_path)
    
    # pdfplumber
    pdfplumber_ok = True
    try:
        extractor2 = PdfPlumberExtractor(calculate_tokens=False)
        result2 = await extractor2.extract(pdf_path)
    except ImportError:
        pdfplumber_ok = False
        print("⚠️  pdfplumber no disponible\n")
    
    # Tabla comparativa
    print("📊 COMPARACIÓN DE EXTRACTORES\n")
    print(f"{'Métrica':<30} {'PyPDF2':<25} {'pdfplumber':<25}")
    print("=" * 80)
    
    if result1.success:
        print(f"{'Success':<30} {str(result1.success):<25} {str(result2.success) if pdfplumber_ok else 'N/A':<25}")
        print(f"{'Páginas':<30} {result1.stats.total_pages:<25} {result2.stats.total_pages if pdfplumber_ok else 'N/A':<25}")
        print(f"{'Caracteres':<30} {f'{result1.stats.total_chars:,}':<25} {f'{result2.stats.total_chars:,}' if pdfplumber_ok else 'N/A':<25}")
        print(f"{'Duración (ms)':<30} {f'{result1.stats.extraction_duration_ms:.2f}':<25} {f'{result2.stats.extraction_duration_ms:.2f}' if pdfplumber_ok else 'N/A':<25}")
        
        if pdfplumber_ok and result2.success:
            # Calcular diferencias
            char_diff = abs(result1.stats.total_chars - result2.stats.total_chars)
            char_diff_pct = (char_diff / result1.stats.total_chars) * 100
            
            print(f"\n📈 ANÁLISIS:")
            print(f"   • Diferencia en caracteres: {char_diff:,} ({char_diff_pct:.1f}%)")
            
            if result1.stats.extraction_duration_ms < result2.stats.extraction_duration_ms:
                ratio = result2.stats.extraction_duration_ms / result1.stats.extraction_duration_ms
                print(f"   • PyPDF2 es {ratio:.2f}x más rápido")
            else:
                ratio = result1.stats.extraction_duration_ms / result2.stats.extraction_duration_ms
                print(f"   • pdfplumber es {ratio:.2f}x más rápido")
    else:
        print(f"❌ Error en extracción PyPDF2: {result1.error}")
else:
    print("⚠️  No hay PDFs disponibles")


🧪 Test 7: Comparación PyPDF2 vs pdfplumber

📄 Comparando con: 20250327_2_Secc.pdf

📊 COMPARACIÓN DE EXTRACTORES

Métrica                        PyPDF2                    pdfplumber               
Success                        True                      True                     
Páginas                        79                        79                       
Caracteres                     630,580                   616,325                  
Duración (ms)                  2900.75                   17037.88                 

📈 ANÁLISIS:
   • Diferencia en caracteres: 14,255 (2.3%)
   • PyPDF2 es 5.87x más rápido


In [11]:
# Test 8: Detección de secciones
print("\n🧪 Test 8: Detección automática de secciones\n")

if test_pdfs:
    pdf_path = test_pdfs[0]
    print(f"📄 PDF: {pdf_path.name}\n")
    
    extractor = PyPDF2Extractor()
    result = await extractor.extract(pdf_path, detect_sections=True)
    
    if result.success:
        print(f"✅ Secciones detectadas: {len(result.sections)}")
        print(f"   - Total páginas: {result.stats.total_pages}\n")
        
        # Mostrar primeras 5 secciones
        for i, section in enumerate(result.sections[:5], 1):
            print(f"📑 Sección {i}:")
            print(f"   - Tipo: {section.section_type.value.upper()}")
            print(f"   - Páginas: {section.start_page} a {section.end_page}")
            print(f"   - Longitud: {len(section.content):,} caracteres")
            preview = section.content[:120].replace('\n', ' ').strip()
            print(f"   - Preview: {preview}...\n")
        
        if len(result.sections) > 5:
            print(f"   ... y {len(result.sections) - 5} secciones más\n")
        
        # Distribución de tipos
        from collections import Counter
        type_dist = Counter(s.section_type.value for s in result.sections)
        print(f"📊 Distribución de tipos de sección:")
        for tipo, count in type_dist.most_common():
            print(f"   • {tipo}: {count}")
    else:
        print(f"❌ Error: {result.error}")
else:
    print("⚠️  No hay PDFs disponibles")


🧪 Test 8: Detección automática de secciones

📄 PDF: 20250327_2_Secc.pdf

✅ Secciones detectadas: 44
   - Total páginas: 79

📑 Sección 1:
   - Tipo: LICITACION
   - Páginas: 1 a 1
   - Longitud: 6,648 caracteres
   - Preview: SUMARIOSECCION1 BOLETIN OFICIAL DE LA PROVINCIA DE CORDOBAJUEVES 27 DE MARZO DE 2025 AÑO CXII - TOMO DCCXXIII - Nº 61 C...

📑 Sección 2:
   - Tipo: GENERAL
   - Páginas: 2 a 2
   - Longitud: 8,039 caracteres
   - Preview: 2 BOLETIN OFICIAL DE LA PROVINCIA DE CORDOBAAÑO CXII - TOMO DCCXXIII - Nº 61 CORDOBA, (R.A.), JUEVES 27 DE MARZO DE 2025...

📑 Sección 3:
   - Tipo: NOMBRAMIENTO
   - Páginas: 3 a 3
   - Longitud: 8,398 caracteres
   - Preview: 3 BOLETIN OFICIAL DE LA PROVINCIA DE CORDOBAAÑO CXII - TOMO DCCXXIII - Nº 61 CORDOBA, (R.A.), JUEVES 27 DE MARZO DE 2025...

📑 Sección 4:
   - Tipo: GENERAL
   - Páginas: 4 a 4
   - Longitud: 7,657 caracteres
   - Preview: 4 BOLETIN OFICIAL DE LA PROVINCIA DE CORDOBAAÑO CXII - TOMO DCCXXIII - Nº 61 CORDOBA, (R.A.), JUEVES 

In [12]:
# Test 9: ExtractorRegistry - Extracción con default
print("\n🧪 Test 9: Extracción usando ExtractorRegistry (default)\n")

if test_pdfs:
    pdf_path = test_pdfs[0]
    
    result = await ExtractorRegistry.extract(pdf_path)
    
    print(f"✅ Extracción con registry exitosa")
    print(f"   - Método usado: {result.stats.extraction_method.value}")
    print(f"   - Success: {result.success}")
    print(f"   - Páginas: {result.stats.total_pages}")
    print(f"   - Caracteres: {result.stats.total_chars:,}")
else:
    print("⚠️  No hay PDFs disponibles")


🧪 Test 9: Extracción usando ExtractorRegistry (default)

✅ Extracción con registry exitosa
   - Método usado: pdfplumber
   - Success: True
   - Páginas: 79
   - Caracteres: 616,325


In [13]:
# Test 10: Helper extract_pdf()
print("\n🧪 Test 10: Función helper extract_pdf()\n")

if test_pdfs:
    pdf_path = test_pdfs[0]
    
    result = await extract_pdf(pdf_path)
    
    print(f"✅ extract_pdf() funcionando")
    print(f"   - Success: {result.success}")
    print(f"   - Método: {result.stats.extraction_method.value}")
    
    print(f"\n💡 API simplificada para uso en producción:")
    print(f"   from app.services.extractors import extract_pdf")
    print(f"   result = await extract_pdf(Path('boletin.pdf'))")
else:
    print("⚠️  No hay PDFs disponibles")


🧪 Test 10: Función helper extract_pdf()

✅ extract_pdf() funcionando
   - Success: True
   - Método: pdfplumber

💡 API simplificada para uso en producción:
   from app.services.extractors import extract_pdf
   result = await extract_pdf(Path('boletin.pdf'))


In [14]:
# Test 11: Selección específica de método
print("\n🧪 Test 11: Forzar método específico\n")

if test_pdfs and len(test_pdfs) > 1:
    pdf_path = test_pdfs[1]
    print(f"📄 PDF: {pdf_path.name}\n")
    
    # Forzar PyPDF2
    result_pypdf2 = await ExtractorRegistry.extract(pdf_path, method="pypdf2")
    print(f"✅ Con PyPDF2 forzado:")
    print(f"   - Método: {result_pypdf2.stats.extraction_method.value}")
    print(f"   - Duración: {result_pypdf2.stats.extraction_duration_ms:.2f} ms")
    
    # Forzar pdfplumber (si está disponible)
    if "pdfplumber" in ExtractorRegistry.list_extractors():
        result_pdfplumber = await ExtractorRegistry.extract(pdf_path, method="pdfplumber")
        print(f"\n✅ Con pdfplumber forzado:")
        print(f"   - Método: {result_pdfplumber.stats.extraction_method.value}")
        print(f"   - Duración: {result_pdfplumber.stats.extraction_duration_ms:.2f} ms")
elif test_pdfs:
    print("ℹ️  Solo hay 1 PDF disponible, usando el mismo para ambos métodos")
    pdf_path = test_pdfs[0]
    
    result_pypdf2 = await ExtractorRegistry.extract(pdf_path, method="pypdf2")
    print(f"✅ PyPDF2: {result_pypdf2.stats.extraction_duration_ms:.2f} ms")
else:
    print("⚠️  No hay PDFs disponibles")


🧪 Test 11: Forzar método específico

📄 PDF: 20250306_5_Secc.pdf

✅ Con PyPDF2 forzado:
   - Método: pypdf2
   - Duración: 381.65 ms

✅ Con pdfplumber forzado:
   - Método: pdfplumber
   - Duración: 2200.60 ms


---
## 🔄 Compatibilidad con Código Legacy

In [15]:
# Test 12: Wrappers deprecated (versiones async para notebooks)
print("🧪 Test 12: Wrappers de compatibilidad (deprecated)\n")

if test_pdfs:
    pdf_path = test_pdfs[0]
    print(f"📄 Testing wrappers con: {pdf_path.name}\n")
    
    # PDFProcessor (versión async)
    from app.services.pdf_service import PDFProcessor
    processor = PDFProcessor()
    text = await processor._extract_text_from_pdf_async(pdf_path)
    print(f"✅ PDFProcessor wrapper (async):")
    print(f"   - Extraído: {len(text):,} caracteres")
    print(f"   - Usa ExtractorRegistry internamente")
    
    # DocumentProcessor (versión async)
    from app.services.document_processor import DocumentProcessor
    doc_processor = DocumentProcessor()
    result = await doc_processor.extract_text_from_pdf_async(str(pdf_path))
    print(f"\n✅ DocumentProcessor wrapper (async):")
    print(f"   - Success: {result['success']}")
    print(f"   - Páginas: {result.get('num_pages', 'N/A')}")
    print(f"   - Formato legacy compatible")
    
    # ContentExtractor (ya es async)
    from app.services.content_extractor import ContentExtractor
    content_extractor = ContentExtractor()
    sections = await content_extractor.extract_from_pdf(pdf_path)
    print(f"\n✅ ContentExtractor wrapper:")
    print(f"   - Secciones: {len(sections)}")
    print(f"   - Formato legacy compatible")
    
    print("\n" + "="*70)
    print("⚠️  TODOS LOS WRAPPERS FUNCIONAN PERO ESTÁN DEPRECADOS")
    print("   → Usar ExtractorRegistry en código nuevo")
    print("   → En notebooks: usar métodos con sufijo _async()")
    print("="*70)
else:
    print("⚠️  No hay PDFs disponibles")

🧪 Test 12: Wrappers de compatibilidad (deprecated)

📄 Testing wrappers con: 20250327_2_Secc.pdf



/var/folders/ft/_5s_0fjd7t9891q188201k300000gn/T/ipykernel_58992/2646039589.py:9: DeprecationWarning: pdf_service.PDFProcessor está deprecado. Usar ExtractorRegistry de app.services.extractors en su lugar.
  from app.services.pdf_service import PDFProcessor


✅ PDFProcessor wrapper (async):
   - Extraído: 616,325 caracteres
   - Usa ExtractorRegistry internamente


/var/folders/ft/_5s_0fjd7t9891q188201k300000gn/T/ipykernel_58992/2646039589.py:17: DeprecationWarning: document_processor.DocumentProcessor está deprecado para extracción. Usar ExtractorRegistry de app.services.extractors en su lugar.
  from app.services.document_processor import DocumentProcessor



✅ DocumentProcessor wrapper (async):
   - Success: True
   - Páginas: 79
   - Formato legacy compatible


/var/folders/ft/_5s_0fjd7t9891q188201k300000gn/T/ipykernel_58992/2646039589.py:26: DeprecationWarning: content_extractor.ContentExtractor está deprecado. Usar ExtractorRegistry.extract(path, detect_sections=True) en su lugar.
  from app.services.content_extractor import ContentExtractor



✅ ContentExtractor wrapper:
   - Secciones: 40
   - Formato legacy compatible

⚠️  TODOS LOS WRAPPERS FUNCIONAN PERO ESTÁN DEPRECADOS
   → Usar ExtractorRegistry en código nuevo
   → En notebooks: usar métodos con sufijo _async()


---
## 🎨 Test Avanzado: Múltiples Documentos

In [16]:
# Test 13: Procesar múltiples PDFs
print("🧪 Test 13: Procesamiento batch de múltiples PDFs\n")

if test_pdfs and len(test_pdfs) >= 2:
    print(f"📦 Procesando {len(test_pdfs)} PDFs...\n")
    
    results = []
    
    for i, pdf_path in enumerate(test_pdfs, 1):
        print(f"🔄 [{i}/{len(test_pdfs)}] Extrayendo {pdf_path.name}...")
        result = await ExtractorRegistry.extract(pdf_path, detect_sections=True)
        results.append((pdf_path.name, result))
        
        if result.success:
            print(f"   ✅ {result.stats.total_pages} pág, {result.stats.total_chars:,} chars, {len(result.sections)} secciones")
        else:
            print(f"   ❌ Error: {result.error}")
    
    # Estadísticas consolidadas
    successful = [r for _, r in results if r.success]
    
    if successful:
        print(f"\n📊 ESTADÍSTICAS CONSOLIDADAS:\n")
        print(f"   • Documentos procesados: {len(successful)}/{len(results)}")
        print(f"   • Total páginas: {sum(r.stats.total_pages for r in successful)}")
        print(f"   • Total caracteres: {sum(r.stats.total_chars for r in successful):,}")
        print(f"   • Total secciones: {sum(len(r.sections) for r in successful)}")
        print(f"   • Tiempo promedio: {sum(r.stats.extraction_duration_ms for r in successful) / len(successful):.2f} ms")
        
        # Distribución de métodos usados
        from collections import Counter
        methods = Counter(r.stats.extraction_method.value for r in successful)
        print(f"\n   • Métodos usados:")
        for method, count in methods.items():
            print(f"     - {method}: {count}")
elif test_pdfs:
    print("ℹ️  Solo hay 1 PDF disponible, test batch omitido")
else:
    print("⚠️  No hay PDFs disponibles")

🧪 Test 13: Procesamiento batch de múltiples PDFs

📦 Procesando 5 PDFs...

🔄 [1/5] Extrayendo 20250327_2_Secc.pdf...
   ✅ 79 pág, 616,325 chars, 40 secciones
🔄 [2/5] Extrayendo 20250306_5_Secc.pdf...
   ✅ 8 pág, 36,222 chars, 5 secciones
🔄 [3/5] Extrayendo 20250306_4_Secc.pdf...
   ✅ 8 pág, 43,600 chars, 1 secciones
🔄 [4/5] Extrayendo 20250327_3_Secc.pdf...
   ✅ 35 pág, 270,313 chars, 15 secciones
🔄 [5/5] Extrayendo 20250331_4_Secc.pdf...
   ✅ 17 pág, 112,548 chars, 1 secciones

📊 ESTADÍSTICAS CONSOLIDADAS:

   • Documentos procesados: 5/5
   • Total páginas: 147
   • Total caracteres: 1,079,008
   • Total secciones: 62
   • Tiempo promedio: 6648.85 ms

   • Métodos usados:
     - pdfplumber: 5


---
## 📊 Resumen Final de Validación

In [17]:
# Resumen consolidado
print("\n" + "═" * 70)
print("📋 RESUMEN DE VALIDACIÓN - ÉPICA 2: EXTRACCIÓN")
print("═" * 70)

print("\n✅ TAREAS IMPLEMENTADAS:\n")
print("   1. Tarea 2.3: Modelo ExtractedContent unificado")
print("      → app/schemas/extraction.py")
print("      → 5 clases Pydantic + 2 enums")

print("\n   2. Tarea 2.1: Extractores unificados")
print("      → app/services/extractors/base.py (ABC)")
print("      → app/services/extractors/pypdf2_extractor.py")
print("      → app/services/extractors/pdfplumber_extractor.py")
print("      → Detección de secciones integrada")

print("\n   3. Tarea 2.2: ExtractorRegistry (Strategy Pattern)")
print("      → app/services/extractors/registry.py")
print("      → Auto-registro de extractores")
print("      → Configuración vía PDF_EXTRACTOR env")
print("      → 6 consumidores refactorizados")

print("\n📈 CONFIGURACIÓN ACTUAL:\n")
print(f"   • Extractores: {ExtractorRegistry.list_extractors()}")
print(f"   • Default: {ExtractorRegistry.get_default()}")
print(f"   • PDFs testeados: {len(test_pdfs)}")

print("\n🎯 ESTADO EN NOTION:\n")
print("   ✅ 2.1 Unificar extractores → HECHO")
print("   ✅ 2.2 ExtractorRegistry → HECHO")
print("   ✅ 2.3 Modelo ExtractedContent → HECHO")

print("\n💡 USO EN PRODUCCIÓN:\n")
print("   from app.services.extractors import extract_pdf")
print("   result = await extract_pdf(Path('documento.pdf'))")
print("   if result.success:")
print("       print(f'Extraídas {result.stats.total_pages} páginas')")

print("\n" + "═" * 70)
print("🎉 ÉPICA 2: EXTRACCIÓN - COMPLETADA Y VALIDADA")
print("═" * 70)
print(f"\nFecha: {datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')} UTC")
print("Estado: ✅ PRODUCCIÓN READY")


══════════════════════════════════════════════════════════════════════
📋 RESUMEN DE VALIDACIÓN - ÉPICA 2: EXTRACCIÓN
══════════════════════════════════════════════════════════════════════

✅ TAREAS IMPLEMENTADAS:

   1. Tarea 2.3: Modelo ExtractedContent unificado
      → app/schemas/extraction.py
      → 5 clases Pydantic + 2 enums

   2. Tarea 2.1: Extractores unificados
      → app/services/extractors/base.py (ABC)
      → app/services/extractors/pypdf2_extractor.py
      → app/services/extractors/pdfplumber_extractor.py
      → Detección de secciones integrada

   3. Tarea 2.2: ExtractorRegistry (Strategy Pattern)
      → app/services/extractors/registry.py
      → Auto-registro de extractores
      → Configuración vía PDF_EXTRACTOR env
      → 6 consumidores refactorizados

📈 CONFIGURACIÓN ACTUAL:

   • Extractores: ['pypdf2', 'pdfplumber']
   • Default: pdfplumber
   • PDFs testeados: 5

🎯 ESTADO EN NOTION:

   ✅ 2.1 Unificar extractores → HECHO
   ✅ 2.2 ExtractorRegistry → HECH